# Qontrol and Picoscope IF - onda triangolare

Il programma controlla in tensione tramite Qontrol i canali specificati nella lista "channels" con uno sweep da "voltage_start'' a ''voltage_stop'' per poi tornare a "voltage_start'', legge le rispettive tensioni e correnti, le salva su file e le visualizza a video con grafici. Inoltre legge la tensione tramite Picoscope del canale specificato (solitamente il fotodiodo), lo plotta e lo salva nel medesimo file.  

## 1) Import

In [1]:
# Required imports
import sys
username = 'admin' # 'admin' for BO1 lab
sys.path.append('c:/users/'+ username +'/miniconda3/lib/site-packages')
import qontrol, os, time, datetime, picosdk, logging, matplotlib.pyplot as plt, \
numpy as np, os.path, ctypes, math
from picosdk.ps4000 import ps4000 as ps
from picosdk.functions import adc2mV, assert_pico_ok
from statistics import mean

%matplotlib inline

# a little function for rounding half up
def round_half_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n*multiplier + 0.5) / multiplier;

#Configure the logging
now = datetime.datetime.now() # retrieve current date and time (for file name)
log_path = './'
os.chdir(log_path)
current_time = now.strftime("%Y_%m_%d_%H_%M_%S")
if not (os.path.isdir('./Log/'+ now.strftime("%Y_%m_%d"))):
    os.makedirs('./Log/'+ now.strftime("%Y_%m_%d"))
log_file_name =  './Log/' + now.strftime("%Y_%m_%d") + '/' + current_time +'.log' 
logging_level = logging.INFO #.WARNING for avoiding info output
logging.basicConfig(filename = log_file_name, level=logging.DEBUG)
console = logging.StreamHandler()
console.setLevel(logging_level)
logging.getLogger().addHandler(console)

logging.info("Section ended")

Section ended


## 2) Dichiarazione variabili e setup driver

In [ ]:
# Variable declarations
number_of_channels = 2
channels = [0 for j in range(number_of_channels)] # the channels connected to the device, FIRST ARGUMENT STANDS FOR THE REPEATED CHANNEL
channel_PD = 3 # the channel connected to the photodiode
voltage_PD = 0 # the voltage to be set to the photodiode
voltage_start = [0.0 for j in range(number_of_channels)] # starting voltage values for each channel
voltage_stop = [12.0 for j in range(number_of_channels)] # stopping voltage values for each channel
voltage_step = [0.2 for j in range(number_of_channels)] # voltage steps for each channel
max_current = 50 # current compliance
max_voltage = 12 # voltage compliance
measured_voltage = [[[] for i in range(len(channels))] for j in range(len(channels))] # measured_voltage[channels][channel_under_sweep][measurement]
measured_current = [[[] for i in range(len(channels))] for j in range(len(channels))] # current measurements
PD_voltage = [[] for i in range(len(channels))] # PD voltage measurements
PD_current = [[] for i in range(len(channels))] # PD current measurements
F_move_on = 0 # FLAG : 0 for pausing when a channel sweep is done
F_overlapping_plots = 1 # FLAG : 0 for non overlapping plots, not 0 otherwise 
F_triangular = 1 # FLAG : 0 for ramp sweep, 1 for triangular sweep

# Configuration (Laser and Amplifier)
laser_wavelength = 697
laser_power = 5
amplifier_gain = 40

# Set the right path and file
if not(os.path.isdir('./Data/'+now.strftime("%Y_%m_%d"))):
    os.makedirs(now.strftime("./Data/%Y_%m_%d"))
if not(os.path.isdir('./Figures/'+now.strftime("%Y_%m_%d"))):
    os.makedirs(now.strftime("./Figures/%Y_%m_%d"))
save_path = './Data/' + now.strftime("%Y_%m_%d") #'G:/My Drive/QPX/QPX3/2020_Dati_Misure_BO1/Q8b/'
file_name = "Qontrol_and_Picoscope_IF_" + current_time +'.txt'
completeName = os.path.join(save_path, file_name)
# Build the header of the file as timestamp+configuration+annotation+voltage_parameters+description
timestamp = '%'+now.strftime("%d/%m/%Y %H:%M:%S")
configuration = '\n%Laser Wavelength:' + str(laser_wavelength) + '[nm]\n%Laser Power:' +str(laser_power) + '[mW]\n%Amplifier Gain:' \
              + str(amplifier_gain) + '[dB]'
# LEFT          RIGHT
# 1              1
# 2              2
annotation = '\n%1-2\n'

voltage_parameters = '%' # voltage_parameters are voltage_start, voltage_stop, voltage_step, triangular
first = True
for element in voltage_start:
    if first == True:
        voltage_parameters += str(element)
    else:
        voltage_parameters += '\t' + str(element)
voltage_parameters += '\n%'
first = True
for element in voltage_stop:
    if first == True:
        voltage_parameters += str(element)
    else:
        voltage_parameters += '\t' + str(element)
voltage_parameters += '\n%'
first = True
for element in voltage_step:
    if first == True:
        voltage_parameters += str(element)
    else:
        voltage_parameters += '\t' + str(element)
voltage_parameters += '\n%' + str(F_triangular) + '\n'

description = '%Swipe sulla tensione con onda sinusoidale su un canale, canali rimanenti a 0 V.Chip Q2-2+8'
header = timestamp+configuration+annotation+voltage_parameters+description

# Write the header and the column headers to the file
meas_file = open(completeName, "a")
meas_file.write(header)
meas_file.write("\n%PD at channel " + str(channel_PD) + ": Voltage [mV]\tCurrent [mA]")
for j in range(len(channels)):
    meas_file.write("\tChannel " +str(channels[j]) + " : Voltage [V]\tCurrent [mA]")
meas_file.write("\n")


# Setup Qontroller
serial_port_name = "COM3" #"COM3" #"/dev/tty.usbserial-FT06QAZ5" # name of the USB Serial Port #//./COM11
q = qontrol.QXOutput(serial_port_name = serial_port_name, response_timeout = 0.1)

# Set the compliances
for channel in range(q.n_chs):
    q.imax[channel] = max_current
    q.vmax[channel] = max_voltage

# Video Debugging
logging.info("Qontroller '{:}' initialised with firmware {:} and {:} channels".format(q.device_id, q.firmware, q.n_chs))

# Setup Picoscope
# Create chandle and status ready for use
chandle = ctypes.c_int16()
status = {}

# Open PicoScope 2000 Series device
# Returns handle to chandle for use in future API functions
status["openunit"] = ps.ps4000OpenUnit(ctypes.byref(chandle))
assert_pico_ok(status["openunit"])


enabled = 1
disabled = 0
analogue_offset = 0.0
nextSample = 0
autoStopOuter = False
wasCalledBack = False

# Set up channel A
# handle = chandle
# channel = PS4000_CHANNEL_A = 0
# enabled = 1
# coupling type = PS4000_DC = 1
# range = PS4000_2V = 7
channel_range = ps.PS4000_RANGE['PS4000_5V']
status["setChA"] = ps.ps4000SetChannel(chandle,
                                        ps.PS4000_CHANNEL['PS4000_CHANNEL_A'],
                                        enabled,
                                        1,
                                        channel_range)
assert_pico_ok(status["setChA"])

# Set up channel B
# handle = chandle
# channel = PS4000_CHANNEL_B = 1
# enabled = 1
# coupling type = PS4000_DC = 1
# range = PS4000_2V = 7
status["setChB"] = ps.ps4000SetChannel(chandle,
                                        ps.PS4000_CHANNEL['PS4000_CHANNEL_B'],
                                        enabled,
                                        1,
                                        channel_range)
assert_pico_ok(status["setChB"])

# CHANGE THESE TO ADJUST THE LENGTH OF THE MEASUREMENT ----------------------------------------------------------------
# Size of capture
sizeOfOneBuffer = 50
numBuffersToCapture = 1
# ---------------------------------------------------------------------------------------------------------------------

totalSamples = sizeOfOneBuffer * numBuffersToCapture

# Create buffers ready for assigning pointers for data collection
bufferAMax = np.zeros(shape=sizeOfOneBuffer, dtype=np.int16)
bufferBMax = np.zeros(shape=sizeOfOneBuffer, dtype=np.int16)
# We need a big buffer, not registered with the driver, to keep our complete capture in.
bufferCompleteA = np.zeros(shape=totalSamples, dtype=np.int16)
bufferCompleteB = np.zeros(shape=totalSamples, dtype=np.int16)

memory_segment = 0

def streaming_callback(handle, noOfSamples, startIndex, overflow, triggerAt, triggered, autoStop, param):
    global nextSample, autoStopOuter, wasCalledBack
    wasCalledBack = True
    destEnd = nextSample + noOfSamples
    sourceEnd = startIndex + noOfSamples
    bufferCompleteA[nextSample:destEnd] = bufferAMax[startIndex:sourceEnd]
    bufferCompleteB[nextSample:destEnd] = bufferBMax[startIndex:sourceEnd]
    nextSample += noOfSamples
    if autoStop:
        autoStopOuter = True

# Set data buffer location for data collection from channel A
# handle = chandle
# source = PS4000_CHANNEL_A = 0
# pointer to buffer max = ctypes.byref(bufferAMax)
# pointer to buffer min = ctypes.byref(bufferAMin)
# buffer length = maxSamples
# segment index = 0
# ratio mode = PS4000_RATIO_MODE_NONE = 0
status["setDataBuffersA"] = ps.ps4000SetDataBuffers(chandle,
                                                     ps.PS4000_CHANNEL['PS4000_CHANNEL_A'],
                                                     bufferAMax.ctypes.data_as(ctypes.POINTER(ctypes.c_int16)),
                                                     None,
                                                     sizeOfOneBuffer)
assert_pico_ok(status["setDataBuffersA"])

# Set data buffer location for data collection from channel B
# handle = chandle
# source = PS4000_CHANNEL_B = 1
# pointer to buffer max = ctypes.byref(bufferBMax)
# pointer to buffer min = ctypes.byref(bufferBMin)
# buffer length = maxSamples
# segment index = 0
# ratio mode = PS4000_RATIO_MODE_NONE = 0
status["setDataBuffersB"] = ps.ps4000SetDataBuffers(chandle,
                                                     ps.PS4000_CHANNEL['PS4000_CHANNEL_B'],
                                                     bufferBMax.ctypes.data_as(ctypes.POINTER(ctypes.c_int16)),
                                                     None,
                                                     sizeOfOneBuffer)
assert_pico_ok(status["setDataBuffersB"])

# Begin streaming mode:
sampleInterval = ctypes.c_int32(250)
sampleUnits = ps.PS4000_TIME_UNITS['PS4000_US']
# We are not triggering:
maxPreTriggerSamples = 0
autoStopOn = 1
# No downsampling:
downsampleRatio = 1

actualSampleInterval = sampleInterval.value
actualSampleIntervalNs = actualSampleInterval * 1000
totalSamplingTime = totalSamples * actualSampleIntervalNs
logging.info("Capturing at sample interval %10.3E ns, with total sampling time of %10.3E ns" % (actualSampleIntervalNs, totalSamplingTime))

def pico_acquire_measurement(discarded_portion = 0.0, plot = False):    
    global status
    
    status["runStreaming"] = ps.ps4000RunStreaming(chandle,
                                                ctypes.byref(sampleInterval),
                                                sampleUnits,
                                                maxPreTriggerSamples,
                                                totalSamples,
                                                autoStopOn,
                                                downsampleRatio,
                                                sizeOfOneBuffer)
    assert_pico_ok(status["runStreaming"])
    
    # We need a big buffer, not registered with the driver, to keep our complete capture in.
    global bufferCompleteA 
    global bufferCompleteB 
    global nextSample 
    global autoStopOuter 
    global wasCalledBack 
    global cFuncPtr 
    
    bufferCompleteA = np.zeros(shape=totalSamples, dtype=np.int16)
    bufferCompleteB = np.zeros(shape=totalSamples, dtype=np.int16)
    nextSample = 0
    autoStopOuter = False
    wasCalledBack = False
    cFuncPtr = ps.StreamingReadyType(streaming_callback)
    
    while nextSample < totalSamples and not autoStopOuter:
        wasCalledBack = False
        status["getStreamingLastestValues"] = ps.ps4000GetStreamingLatestValues(chandle, cFuncPtr, None)
        if not wasCalledBack:
            # If we weren't called back by the driver, this means no data is ready. Sleep for a short while before trying
            # again.
            time.sleep(0.01)
    
    #logging.info("Done grabbing values.")
    
    # Find maximum ADC count value
    # handle = chandle
    # pointer to value = ctypes.byref(maxADC)
    maxADC = ctypes.c_int16(32767)

    # Convert ADC counts data to mV
    adc2mVChAMax = adc2mV(bufferCompleteA, channel_range, maxADC)
    #adc2mVChBMax = adc2mV(bufferCompleteB, channel_range, maxADC)
    
    # Stop the scope
    # handle = chandle
    status["stop"] = ps.ps4000Stop(chandle)
    assert_pico_ok(status["stop"])
    
    if plot:
        # Plot data from channel A
        # Create time data
        time_axis = np.linspace(0, (totalSamples) * actualSampleIntervalNs, totalSamples)
        plt.plot(time_axis, adc2mVChAMax[:])
        #plt.plot(time, adc2mVChBMax[:])
        plt.xlabel('Time (ns)')
        plt.ylabel('Voltage (mV)')
        plt.show()

    return round_half_up(mean(adc2mVChAMax[math.floor((len(adc2mVChAMax)-1)*discarded_portion):]),3);

def pico_stop():
    global status
    
    # Stop the scope
    # handle = chandle
    status["stop"] = ps.ps4000Stop(chandle)
    assert_pico_ok(status["stop"])    

    # Disconnect the scope
    # handle = chandle
    status["close"] = ps.ps4000CloseUnit(chandle)
    assert_pico_ok(status["close"])
    return;

logging.info("Section ended")

## 3) Acquisizione e visualizzazione dati

In [ ]:
# PD voltage setting
q.v[channel_PD] = voltage_PD

#Set current for channel K
#q.i[K] = I

while F_move_on != 1: # wait for user input to move on
    F_move_on = int(input("Do you want to move on? No -> 0 Yes -> 1\n"))    


# Perform a voltage sweep for each channel    
for channel_n in range(len(channels)):
    
    F_move_on = 0
    iteration = 0
    while F_move_on != 1: # wait for user input to move on
        F_move_on = int(input("Do you want to move to channel " + str(channels[channel_n]) + "? No -> 0 Yes -> 1\n"))    
    
    
    print("\nDRIVING CHANNEL {:} NOW \n" .format(channels[channel_n]))
    
    # set all channels to their voltage_start values
    for k in range(len(channels)):
        q.v[channels[k]] = float(voltage_start[k])
    
    #check if we want to generate a triangular sweep or not
    if F_triangular == 1:
        sweep_range = np.concatenate((np.arange(voltage_start[channel_n], voltage_stop[channel_n], voltage_step[channel_n]),\
                        np.arange(voltage_stop[channel_n], voltage_start[channel_n], -voltage_step[channel_n])))
    elif F_triangular == 0:
        sweep_range = np.arange(voltage_start[0], voltage_stop[0]+voltage_step[0], voltage_step[0])
    else:
        print('invalid value for triangular')
    
    for voltage_sweep in sweep_range: 
        q.v[channels[channel_n]] = float(voltage_sweep)
        # CHANGE THE DISCARDED PORTION TO DISCARD A SECTION OF THE AVERAGED MEASUREMENTS OF THE PICOSCOPE ----------------
        temp_voltage = pico_acquire_measurement(discarded_portion = 0.0)
        # ----------------------------------------------------------------------------------------------------------------
        temp_current = -q.i[channel_PD]
        #temp_voltage = q.v[channel_PD]
        #temp_current = -q.i[channel_PD]
        PD_voltage[channel_n].append(temp_voltage)
        PD_current[channel_n].append(temp_current)
        #print("PD [ch {:}] : Voltage -> {:} V, Current -> {:} mA" .format(channel_PD, temp_voltage, temp_current))
        meas_file.write('{:+010.6f}\t{:+010.6f}' .format(temp_voltage,  temp_current))
        
        for j in range(len(channels)): # acquire voltage and current for all channels. Then print them on screen and save them on file
            temp_voltage = q.v[channels[j]]
            temp_current = q.i[channels[j]]
            measured_voltage[j][channel_n].append(temp_voltage)
            measured_current[j][channel_n].append(temp_current)
            #print("Channel {:} : Voltage -> {:} V, Current -> {:} mA" .format(channels[j], temp_voltage, temp_current))
            meas_file.write("\t{:+010.6f}\t{:+010.6f}" .format(temp_voltage, temp_current))
        meas_file.write("\n")
        iteration+=1
        logging.info("Percentage of completion: {} %" .format(round_half_up(iteration/len(sweep_range)*100, 1)))
    
    # at the end of each sweep, set all channels to their voltage_start values
    for k in range(len(channels)):
        q.v[channels[k]] = float(voltage_start[k])
    
    
    


# Plot results
fig, axs = plt.subplots(2,1)

axs[0].set_title('Channel Voltage vs  Channel Current ')
axs[0].set_xlabel('Voltage [V]')
axs[0].set_ylabel('Current [mA]')
axs[1].set_title('Channel Voltage vs  -PD Voltage ')
axs[1].set_xlabel('Voltage [V]')
axs[1].set_ylabel('Voltage [mV]')

for i in range(len(channels)):
    #print("\nPLOTS FOR DRIVING CHANNEL {:} \n" .format(channels[i]))
    if F_overlapping_plots == 0:
        axs[0].plot(range(i*len(measured_current[i][i]),(i+1)*len(measured_current[i][i])), measured_current[i][i])
        axs[1].plot(range(i*len(measured_current[i][i]),(i+1)*len(measured_current[i][i])), PD_voltage[i])
    else:
        axs[0].plot(measured_voltage[i][i], measured_current[i][i])
        axs[1].plot(range(len(measured_current[i][i])), PD_voltage[i])
plt.tight_layout()
plt.savefig(now.strftime("./Figures/%Y_%m_%d/") + file_name + '.png')


# Close the log file
meas_file.close()

# Set all channels to zero
q.v[:] = 0
q.i[:] = 0

# Close the communication with the driver
q.close()

# Close the Picoscope
pico_stop()

logging.info("Section ended")